# Using MLRUN with Dask Distributed Jobs

In [1]:
# NOTE: It's recommended that you install the exact package versions:
#!pip install dask==2.9.1 distributed==2.9.1 kubernetes==10.0.0 dask_kubernetes==0.10.0 kubernetes-asyncio==10.0.0 msgpack==0.6.2

## Writing Function Code

In [2]:
# function that will be distributed 
def inc(x):
    return x+2

The MLRun context in the case of Dask will have an extra param `dask_client`
which is initialized based on the function spec (below), and can be used to submit Dask commands.

In [3]:
# wrapper function, uses the dask client object
def hndlr(context, x=1,y=2):
    context.logger.info('params: x={},y={}'.format(x,y))
    print('params: x={},y={}'.format(x,y))
    x = context.dask_client.submit(inc, x)
    print(x)
    print(x.result())
    context.log_result('y', x.result())

In [4]:
# nuclio: end-code
# marks the end of a code section, do not delete

In [5]:
# nuclio: ignore
import nuclio

In [6]:
from mlrun import new_function, mlconf, code_to_function, mount_v3io, NewTask
mlconf.dbpath = 'http://mlrun-api:8080'

## Define a Dask Function Object
Dask functions can be local (local workers) or remote (using containers on a remote cluster).
For remote functions, you can optionally select whether to specify the number of replicas or leave this configuration empty for auto-scaling.

The following example uses the `code_to_function` method to pack the function code from the web notebook into a function object (a YAML configuration file), thus eliminating the need to update the function image.
Alternatively, you can use the `new_function` method if the function code is part of the image or can be remotely mounted (for example, via the `v3io` mount).

The Dask function specification has several unique attributes:
- **`.remote`** (`bool`) &mdash; `True` to use a remote or clustered Dask function; `False` (default) otherwise.
- **`.replicas`** &mdash; the number of desired replicas; 0 is used for auto-scaling.
- **`.min_replicas`, `.max_replicas`** &mdash; the minimum and maximum number of replicas for auto-scaling.
- **`.scheduler_timeout`** &mdash; an inactivity timeout period after which the cluster will be killed; the default is `'60 minutes'` (1 hour).
* **`.nthreads`** &mdash; the number of worker threads to use.
* **`.kfp_image`** &mdash; the name of the container image to use by the KFP pipeline runner (default: `'mlrun/dask'`).

If you want to access the Dask dashboard remotely, you need to use the `NodePort` service type by setting `.service_type` to 'NodePort', and the external IP needs to be specified in the MLRun configuration (`mlconf.remote_host`).
This is set automatically when running on an Iguazio Data Science Platform cluster.
If you want to use the `NodePort` for remote access to the scheduler, you need to also specify `function.use_remote=True`.

In [9]:
# Create an `xgb_train` Nuclio function from the notebook code and annotations;
# add a v3io data volume and a multi-worker HTTP trigger for parallel execution
dsf = code_to_function('mydask', kind='dask').apply(mount_v3io())

In [10]:
dsf.spec.image = 'daskdev/dask:2.9.1'
dsf.spec.remote = True
dsf.spec.replicas = 1
dsf.spec.service_type = 'NodePort'
dsf.spec.image_pull_policy = 'Always'

## Build the Function with Extra Packages
The following code demonstrates how to build the function with extra packages.
You can skip this step if you don't need to add packages.
Instead, specify the image image; for example, `dsf.spec.image='daskdev/dask:2.9.1'`.

In [11]:
# Uncomment to add packages to the workers and build a new image
# dsf.build_config(base_image='daskdev/dask:2.9.1', commands=['pip install pandas'])
# dsf.deploy()

## Run a Task Using a Distributed Dask Function (Cluster)

In [12]:
myrun = dsf.run(handler=hndlr, params={'x': 12})

[mlrun] 2020-02-24 22:23:48,435 starting run mydask-hndlr uid=d724caf66eba4e4baa86faf539c81b53  -> http://mlrun-api:8080
[mlrun] 2020-02-24 22:23:49,032 saving function: mydask, tag: latest
[mlrun] 2020-02-24 22:23:56,887 trying dask client at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786
[mlrun] 2020-02-24 22:23:56,896 using remote dask scheduler (mlrun-mydask-75a63220-9) at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786


[mlrun] 2020-02-24 22:23:56,904 params: x=12,y=2
params: x=12,y=2
<Future: pending, key: inc-d94d5c09f5ea9337e23aad080db4545d>
14

[mlrun] 2020-02-24 22:24:02,571 run ended with state 


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...c81b53,0,Feb 24 22:23:48,completed,mydask-hndlr,kind=daskowner=adminhost=jupyter-68bdf65845-httbr,,x=12,y=14,


To track results, use `.show()` or `.logs()` or this  CLI command:
!mlrun get run d724caf66eba4e4baa86faf539c81b53  , !mlrun logs d724caf66eba4e4baa86faf539c81b53 
[mlrun] 2020-02-24 22:24:02,609 run executed, status=completed


In [13]:
# get the function status and addresses
dsf.status.to_dict()

{'scheduler_address': 'tcp://mlrun-mydask-75a63220-9.default-tenant:8786',
 'cluster_name': 'mlrun-mydask-75a63220-9',
 'node_ports': {'dashboard': 30489, 'scheduler': 31426}}

### Accesing the dask client directly
You can get the dask client object and cluster information by reading the client object.

> Note: the cluster can timeout, when you call the client MLrun will also verify the cluster is live and active and if not it will restart the dask cluster and refresh the function object with the latest addresses and status.

In [14]:
c = dsf.client

[mlrun] 2020-02-24 22:24:10,108 trying dask client at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786
[mlrun] 2020-02-24 22:24:10,115 using remote dask scheduler (mlrun-mydask-75a63220-9) at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786


### Access a dask function using the DB
If we want to access the dask function (or its cluster), we can load the function object from the DB (assuming we already .run() or .save() it).

This can be useful if we want to load the same function in a different notebook or container, or if we restarted our notebook

In [15]:
from mlrun import import_function
# Functions URL: db://<project>/<name>[:<tag>]
dsf_obj = import_function('db://default/mydask')
c = dsf_obj.client

[mlrun] 2020-02-24 22:24:15,282 trying dask client at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786
[mlrun] 2020-02-24 22:24:15,289 using remote dask scheduler (mlrun-mydask-75a63220-9) at: tcp://mlrun-mydask-75a63220-9.default-tenant:8786


## Building a Pipeline using dask functions

In [16]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [17]:
@dsl.pipeline(name="dask_pipeline")
def dask_pipe(x=1,y=10):
    # use_db option will use a function (DB) pointer instead of adding the function spec to the YAML
    myrun = dsf.as_step(NewTask(handler=hndlr, name="dask_pipeline", params={'x': x, 'y': y}), use_db=True)
    # is the step (dask client) need v3io access u should add: .apply(mount_v3io())
    myrun.container.set_image_pull_policy('Always')

In [18]:
# for pipeline debug
kfp.compiler.Compiler().compile(dask_pipe, 'daskpipe.yaml', type_check=False)

[mlrun] 2020-02-24 22:24:38,635 saving function: mydask, tag: latest


/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "10".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [19]:
arguments={'x':4,'y':-5}
artifact_path = '/User/test'
run_pipeline(dask_pipe, arguments, artifact_path=artifact_path,
             run="DaskExamplePipeline", experiment="dask pipe")

[mlrun] 2020-02-24 22:24:49,917 saving function: mydask, tag: latest


[mlrun] 2020-02-24 22:24:49,966 Pipeline run id=4eefbad4-abc9-48aa-9dc6-82c70f968a82, check the MLRun dashboard or DB for progress


'4eefbad4-abc9-48aa-9dc6-82c70f968a82'